In [11]:
import kaggle as kg
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import to_categorical

In [2]:
os.environ["KAGGLE_USERNAME"] = "aimlrl"
os.environ["KAGGLE_KEY"] = "54d4150a6ca782d7b27af3f3754eddd8"

In [3]:
kg.api.authenticate()

In [6]:
kg.api.dataset_download_files(dataset="medahmedkrichen/devanagari-handwritten-character-datase",
                              path="dataset",unzip=True)

Dataset URL: https://www.kaggle.com/datasets/medahmedkrichen/devanagari-handwritten-character-datase


In [13]:
def train_test_df(path):

    img_path = list()
    img_label = list()

    for single_class_dir_path in pathlib.Path(path).glob("*"):

        for single_class_img_path in pathlib.Path(single_class_dir_path).glob("*.png"):

            img_path.append(str(single_class_img_path))
            #print(str(single_class_img_path).split("/")[-2].split("_")[-1])
            img_label.append(str(single_class_img_path).split("/")[-2].split("_")[-1])

    return pd.DataFrame(data={"img_path":img_path,"label":img_label})        

In [14]:
train_path = "dataset/DevanagariHandwrittenCharacterDataset/Train"
test_path = "dataset/DevanagariHandwrittenCharacterDataset/Test"

In [15]:
training_data = train_test_df(train_path)
testing_data = train_test_df(test_path)

In [16]:
training_data

,img_path,label
0,dataset/DevanagariHandwrittenCharacterDataset/...,ga
1,dataset/DevanagariHandwrittenCharacterDataset/...,ga
2,dataset/DevanagariHandwrittenCharacterDataset/...,ga
3,dataset/DevanagariHandwrittenCharacterDataset/...,ga
4,dataset/DevanagariHandwrittenCharacterDataset/...,ga
...,...,...
78195,dataset/DevanagariHandwrittenCharacterDataset/...,ja
78196,dataset/DevanagariHandwrittenCharacterDataset/...,ja
78197,dataset/DevanagariHandwrittenCharacterDataset/...,ja
78198,dataset/DevanagariHandwrittenCharacterDataset/...,ja


In [17]:
testing_data

,img_path,label
0,dataset/DevanagariHandwrittenCharacterDataset/...,ga
1,dataset/DevanagariHandwrittenCharacterDataset/...,ga
2,dataset/DevanagariHandwrittenCharacterDataset/...,ga
3,dataset/DevanagariHandwrittenCharacterDataset/...,ga
4,dataset/DevanagariHandwrittenCharacterDataset/...,ga
...,...,...
13795,dataset/DevanagariHandwrittenCharacterDataset/...,ja
13796,dataset/DevanagariHandwrittenCharacterDataset/...,ja
13797,dataset/DevanagariHandwrittenCharacterDataset/...,ja
13798,dataset/DevanagariHandwrittenCharacterDataset/...,ja


In [18]:
character2int = dict(zip(training_data["label"].unique(),range(len(training_data["label"].unique()))))

In [19]:
character2int

{'ga': 0,
 'yna': 1,
 'la': 2,
 '8': 3,
 'dha': 4,
 'ma': 5,
 '1': 6,
 'petchiryakha': 7,
 'taamatar': 8,
 'cha': 9,
 'bha': 10,
 'gha': 11,
 'tabala': 12,
 '7': 13,
 'yaw': 14,
 'daa': 15,
 'thaa': 16,
 '0': 17,
 'tra': 18,
 'chhya': 19,
 '4': 20,
 'na': 21,
 'da': 22,
 '9': 23,
 'patalosaw': 24,
 'pha': 25,
 'jha': 26,
 'waw': 27,
 'tha': 28,
 '3': 29,
 'chha': 30,
 'gya': 31,
 '2': 32,
 'ka': 33,
 'ba': 34,
 'ha': 35,
 'kha': 36,
 'motosaw': 37,
 'dhaa': 38,
 'pa': 39,
 'kna': 40,
 'adna': 41,
 '5': 42,
 '6': 43,
 'ra': 44,
 'ja': 45}

In [20]:
training_data["label"].replace(to_replace=character2int.keys(),value=character2int.values(),
                               inplace=True)

/tmp/ipykernel_1422/2698354796.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  training_data["label"].replace(to_replace=character2int.keys(),value=character2int.values(),


In [21]:
training_data

,img_path,label
0,dataset/DevanagariHandwrittenCharacterDataset/...,0
1,dataset/DevanagariHandwrittenCharacterDataset/...,0
2,dataset/DevanagariHandwrittenCharacterDataset/...,0
3,dataset/DevanagariHandwrittenCharacterDataset/...,0
4,dataset/DevanagariHandwrittenCharacterDataset/...,0
...,...,...
78195,dataset/DevanagariHandwrittenCharacterDataset/...,45
78196,dataset/DevanagariHandwrittenCharacterDataset/...,45
78197,dataset/DevanagariHandwrittenCharacterDataset/...,45
78198,dataset/DevanagariHandwrittenCharacterDataset/...,45


In [22]:
testing_data.replace(to_replace=character2int.keys(),value=character2int.values(),
                     inplace=True)

/tmp/ipykernel_1422/3185811566.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  testing_data.replace(to_replace=character2int.keys(),value=character2int.values(),


In [23]:
testing_data

,img_path,label
0,dataset/DevanagariHandwrittenCharacterDataset/...,0
1,dataset/DevanagariHandwrittenCharacterDataset/...,0
2,dataset/DevanagariHandwrittenCharacterDataset/...,0
3,dataset/DevanagariHandwrittenCharacterDataset/...,0
4,dataset/DevanagariHandwrittenCharacterDataset/...,0
...,...,...
13795,dataset/DevanagariHandwrittenCharacterDataset/...,45
13796,dataset/DevanagariHandwrittenCharacterDataset/...,45
13797,dataset/DevanagariHandwrittenCharacterDataset/...,45
13798,dataset/DevanagariHandwrittenCharacterDataset/...,45


In [24]:
Y_true_train = to_categorical(y=training_data["label"],num_classes=46)

In [25]:
Y_true_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [26]:
Y_true_test = to_categorical(y=testing_data["label"],num_classes=46)

In [27]:
Y_true_test

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)